In [11]:
# !pip install syft==0.2.9

In [12]:
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import syft as sy
import copy
import numpy as np
import pandas as pd
import time
from sklearn.metrics import classification_report
from sklearn.model_selection import GroupShuffleSplit

In [13]:
class Arguments():
    def __init__(self):
        self.images = 60000
        self.clients = 2
        self.rounds = 1
        self.epochs = 5
        self.local_batches = 64
        self.lr = 0.01
        self.C = 0.9
        self.drop_rate = 0.1
        self.torch_seed = 0
        self.log_interval = 10
        self.iid = 'iid'
        self.split_size = int(self.images / self.clients)
        self.samples = self.split_size / self.images 
        self.use_cuda = False
        self.save_model = False

args = Arguments()

use_cuda = args.use_cuda and th.cuda.is_available()
device = th.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [14]:
hook = sy.TorchHook(th)
clients = []

for i in range(args.clients):
    clients.append({'hook': sy.VirtualWorker(hook, id="client{}".format(i+1))})

In [15]:
path = "/content/drive/MyDrive/Thesis/Datasets/Turbofan_Dataset/final_datasets_normalized/"

In [16]:
class Arguments():
    def __init__(self):
        self.images = 60000
        self.clients = 2
        self.rounds = 5
        self.epochs = 5
        self.local_batches = 64
        self.lr = 0.01
        self.C = 0.9
        self.drop_rate = 0.1
        self.torch_seed = 0
        self.log_interval = 10
        self.iid = 'iid'
        self.split_size = int(self.images / self.clients)
        self.samples = self.split_size / self.images 
        self.use_cuda = False
        self.save_model = False

args = Arguments()

use_cuda = args.use_cuda and th.cuda.is_available()
device = th.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [17]:
def binary_acc(y_pred, y_test):
    y_pred_tag = th.round(th.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = th.round(acc * 100)
    
    return acc

In [18]:
hook = sy.TorchHook(th)
clients = []

for i in range(args.clients):
    clients.append({'hook': sy.VirtualWorker(hook, id="client{}".format(i+1))})

In [19]:
# Load data and drop irrelevant columns

alice_set = pd.read_csv(path + "TRAINING_SET_1.csv")
bob_set = pd.read_csv(path + "TRAINING_SET_2.csv")

test_set = pd.read_csv(path + "TEST_SET_FULL.csv")

drop_cols = ["cycle","setting3","s1","s5","s10","s16","s18","s19","RUL"]
corr_cols = ["s11","s4","s15","s17","s2","s3","s8","s13","s9","s14","s12","s7","s20"]
feature_cols = ['cycle_norm', 'setting1', 'setting2', 's2', 's3', 's4', 's6', 's7',
       's8', 's9', 's11', 's12', 's13', 's14', 's15', 's17', 's20', 's21']
prediction_col = 'fail_30'

alice_set = alice_set.drop(drop_cols, axis=1)
bob_set = bob_set.drop(drop_cols, axis=1)

test_set = test_set.drop(drop_cols, axis=1)

In [20]:
# Receives single engine dataframe, window size and features -> sequences of length==window_size
def gen_train_data(df, sequence_length, columns):
    data = df[columns].values
    num_elements = data.shape[0]

    # -1 and +1 because of Python indexing
    for start, stop in zip(range(0, num_elements-(sequence_length-1)), range(sequence_length, num_elements+1)):
        yield data[start:stop, :]

In [21]:
# Generates sequences for multiple engines
def gen_data_wrapper(df, sequence_length, columns, ids=np.array([])):
    if ids.size <= 0:
        ids = df['id'].unique()
        
    data_gen = (list(gen_train_data(df[df['id']==id], sequence_length, columns))
               for id in ids)
    data_array = np.concatenate(list(data_gen)).astype(np.float32)
    return data_array

In [22]:
# Functions to generate sequences for the labals
def gen_labels(df, sequence_length, label):
    data_matrix = df[label].values
    num_elements = data_matrix.shape[0]

    # -1 because I want to predict the rul of that last row in the sequence, not the next row
    return data_matrix[sequence_length-1:num_elements, :]  

def gen_label_wrapper(df, sequence_length, label, ids=np.array([])):
    if ids.size <= 0:
        ids = df['id'].unique()
        
    label_gen = [gen_labels(df[df['id']==id], sequence_length, label) 
                for id in ids]
    label_array = np.concatenate(label_gen).astype(np.float32)
    return label_array

In [23]:
def gen_test_data(df, sequence_length, columns, mask_value):
    if df.shape[0] < sequence_length:
        data_matrix = np.full(shape=(sequence_length, len(columns)), fill_value=mask_value) # pad
        idx = data_matrix.shape[0] - df.shape[0]
        data_matrix[idx:,:] = df[columns].values  # fill with available data
    else:
        data_matrix = df[columns].values
        
    # specifically yield the last possible sequence
    stop = num_elements = data_matrix.shape[0]
    start = stop - sequence_length
    for i in list(range(1)):
        yield data_matrix[start:stop, :]
def gen_test_label_wrapper(df, sequence_length, label, ids=np.array([])):
    if ids.size <= 0:
        ids = df['id'].unique()
    
    label_gen = [gen_labels(df[df['id']==id], sequence_length, label) 
                for id in ids]
    # keep only last window
    if sequence_length > 31:
      print("Too big window")
    else:
      last_labels = [label[-1] for label in label_gen] 
      



    last_labels = np.concatenate(last_labels).astype(np.float32)
    # return label_array
    return last_labels


In [24]:
# Functions to generate sequences for the labals
def gen_labels(df, sequence_length, label):
    data_matrix = df[label].values
    num_elements = data_matrix.shape[0]

    # -1 because I want to predict the rul of that last row in the sequence, not the next row
    return data_matrix[sequence_length-1:num_elements, :]  

def gen_label_wrapper(df, sequence_length, label, ids=np.array([])):
    if ids.size <= 0:
        ids = df['id'].unique()
        
    label_gen = [gen_labels(df[df['id']==id], sequence_length, label) 
                for id in ids]
    label_array = np.concatenate(label_gen).astype(np.float32)
    return label_array

In [25]:
def gen_test_data(df, sequence_length, columns, mask_value):
    if df.shape[0] < sequence_length:
        data_matrix = np.full(shape=(sequence_length, len(columns)), fill_value=mask_value) # pad
        idx = data_matrix.shape[0] - df.shape[0]
        data_matrix[idx:,:] = df[columns].values  # fill with available data
    else:
        data_matrix = df[columns].values
        
    # specifically yield the last possible sequence
    stop = num_elements = data_matrix.shape[0]
    start = stop - sequence_length
    for i in list(range(1)):
        yield data_matrix[start:stop, :]
def gen_test_label_wrapper(df, sequence_length, label, ids=np.array([])):
    if ids.size <= 0:
        ids = df['id'].unique()
    
    label_gen = [gen_labels(df[df['id']==id], sequence_length, label) 
                for id in ids]
    # keep only last window
    if sequence_length > 31:
      print("Too big window")
    else:
      last_labels = [label[-1] for label in label_gen] 
      
    last_labels = np.concatenate(last_labels).astype(np.float32)
    # return label_array
    return last_labels


In [26]:
sequence_length = 20

gss = GroupShuffleSplit(n_splits=1, train_size=0.80, random_state=42)


for alice_train_unit, alice_val_unit in gss.split(alice_set['id'].unique(), groups=alice_set['id'].unique()):
    alice_train_unit = alice_set['id'].unique()[alice_train_unit]  # gss returns indexes and index starts at 1
    alice_val_unit = alice_set['id'].unique()[alice_val_unit]

    train_split_array = gen_data_wrapper(alice_set, sequence_length, feature_cols, alice_train_unit)
    train_split_label = gen_label_wrapper(alice_set, sequence_length, ['fail_30'], alice_train_unit)
    
    val_split_array = gen_data_wrapper(alice_set, sequence_length, feature_cols, alice_val_unit)
    val_split_label = gen_label_wrapper(alice_set, sequence_length, ['fail_30'], alice_val_unit)

for bob_train_unit, bob_val_unit in gss.split(bob_set['id'].unique(), groups=bob_set['id'].unique()):
    bob_train_unit = bob_set['id'].unique()[bob_train_unit]  # gss returns indexes and index starts at 1
    bob_val_unit = bob_set['id'].unique()[bob_val_unit]

    train_split_array = gen_data_wrapper(bob_set, sequence_length, feature_cols, bob_train_unit)
    train_split_label = gen_label_wrapper(bob_set, sequence_length, ['fail_30'], bob_train_unit)
    
    val_split_array = gen_data_wrapper(bob_set, sequence_length, feature_cols, bob_val_unit)
    val_split_label = gen_label_wrapper(bob_set, sequence_length, ['fail_30'], bob_val_unit)

# create sequences train, test 
X_alice = gen_data_wrapper(alice_set, sequence_length, feature_cols)
X_bob = gen_data_wrapper(bob_set, sequence_length, feature_cols)

y_alice = gen_label_wrapper(alice_set, sequence_length, ['fail_30'])
y_bob = gen_label_wrapper(bob_set, sequence_length, ['fail_30'])


test_gen = (list(gen_test_data(test_set[test_set['id']==id], sequence_length, feature_cols, -99.))
           for id in test_set['id'].unique())
X_test = np.concatenate(list(test_gen)).astype(np.float32)

y_test = gen_test_label_wrapper(test_set, sequence_length, ['fail_30'])


In [27]:
# Defining custom dataset class for convenience

class CustomDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [28]:
alice_dataset = CustomDataset(X_alice, y_alice)
bob_dataset = CustomDataset(X_bob, y_bob)
test_dataset = CustomDataset(X_test, y_test)

In [29]:
alice_loader = DataLoader(dataset=alice_dataset, batch_size=args.local_batches, shuffle=False)
bob_loader = DataLoader(dataset=bob_dataset, batch_size=args.local_batches, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=args.local_batches, shuffle=False)

In [30]:
clients[0]['trainset'] = alice_loader
clients[1]['trainset'] = bob_loader

In [31]:
class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        self.num_features = 18
        self.hidden_units = 32
        self.num_layers = 1

        self.rnn = nn.RNN(
            input_size=self.num_features,
            hidden_size=self.hidden_units,
            batch_first=True,
            num_layers=self.num_layers) 
        
        self.linear = nn.Linear(in_features=self.hidden_units, out_features=1)

        self.tanh = nn.Tanh()
        # self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
      batch_size = x.shape[0]
      h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
      # c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
      _, hn = self.rnn(x, h0)
      out = self.linear(hn[0]).flatten()  # First dim of Hn is num_layers, which is set to 1 above.

      return out
        # x, (hn, cn) = self.lstm1(inputs)
        # x = self.linear(x)
        # x = self.tanh(x) 
        # return x

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
device = th.device("cuda:0" if th.cuda.is_available() else "cpu")
print(device)

In [ ]:
EPOCHS = 5
LEARNING_RATE = 0.001
criterion = nn.BCEWithLogitsLoss()

In [ ]:
th.manual_seed(args.torch_seed)
global_model = BinaryClassification()

for client in clients:
    th.manual_seed(args.torch_seed)
    client['model'] = BinaryClassification().to(device)
    client['optim'] = optim.SGD(client['model'].parameters(), lr=args.lr)

In [ ]:
def binary_acc(y_pred, y_test):
    y_pred_tag = th.round(th.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = th.round(acc * 100)
    
    return acc

In [ ]:
def ClientUpdate(args, device, client):
    client['model'].train()
    client['model'].send(client['hook'])

    for epoch in range(1, args.epochs + 1):

        for batch_idx, (data, target) in enumerate(client['trainset']):
            data = data.send(client['hook'])
            target = target.send(client['hook'])
            
            data, target = data.to(device), target.to(device)
            client['optim'].zero_grad()
            print("before")
            print(data.size(0))
            output = client['model'](data)
            loss = criterion(output, target.unsqueeze(1))
            acc = binary_acc(target, target.unsqueeze(1))
            loss.backward()
            client['optim'].step()


            
            # if batch_idx % args.log_interval == 0:
            #     loss = loss.get()
            #     acc = acc.get() 
                # print('Model {} Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                #     client['hook'].id,
                #     epoch, batch_idx * args.local_batches, len(client['trainset']) * args.local_batches, 
                #     100. * batch_idx / len(client['trainset']), loss))
        # print(f'Epoch {epoch+0:03}: | Loss: {epoch_loss/len(client["trainset"]):.5f} | Acc: {epoch_acc/(args.local_batches * len(client["trainset"])):.3f}')
                
    client['model'].get() 

def averageModels(global_model, clients):
    client_models = [clients[i]['model'] for i in range(len(clients))]
    # samples = [clients[i]['samples'] for i in range(len(clients))]
    global_dict = global_model.state_dict()
    
    for k in global_dict.keys():
        global_dict[k] = th.stack([client_models[i].state_dict()[k].float() * 0.5 for i in range(len(client_models))], 0).sum(0)
        # global_dict[k] = th.stack([client_models[i].state_dict()[k].float() * samples[i] for i in range(len(client_models))], 0).sum(0)
        

    global_model.load_state_dict(global_dict)

    # for i in range(len(client_models)):
    #   print(f"model {i+1}")
    #   print(client_models[i].state_dict())
    # print("global model")
    # print(global_model.state_dict())
    return global_model


In [ ]:
for fed_round in range(args.rounds):
    
#     uncomment if you want a randome fraction for C every round
#     args.C = float(format(np.random.random(), '.1f'))
    
    print(f"Federated round {fed_round+1}")
    active_clients = clients
    
    # Training 
    for client in active_clients:
        ClientUpdate(args, device, client)
    
#     # Testing 
#     for client in active_clients:
#         test(args, client['model'], device, client['testset'], client['hook'].id)
    
    # Averaging 
    global_model = averageModels(global_model, active_clients)
    
    # Testing the average model
    # test(args, global_model, device, global_test_loader, 'Global')
            
    # Share the global model with the clients
    for client in clients:
        client['model'].load_state_dict(global_model.state_dict())
        
# if (args.save_model):
#     torch.save(global_model.state_dict(), "FedAvg.pt")

In [ ]:
th.manual_seed(args.torch_seed)
global_model = BinaryClassification()

for client in clients:
    th.manual_seed(args.torch_seed)
    client['model'] = BinaryClassification().to(device)
    client['optim'] = optim.SGD(client['model'].parameters(), lr=args.lr)

In [ ]:
for fed_round in range(args.rounds):
    
#     uncomment if you want a randome fraction for C every round
#     args.C = float(format(np.random.random(), '.1f'))
    
    print(f"Federated round {fed_round+1}")
    active_clients = clients
    
    # Training 
    for client in active_clients:
        ClientUpdate(args, device, client)
    
#     # Testing 
#     for client in active_clients:
#         test(args, client['model'], device, client['testset'], client['hook'].id)
    
    # Averaging 
    global_model = averageModels(global_model, active_clients)
    
    # Testing the average model
    # test(args, global_model, device, global_test_loader, 'Global')
            
    # Share the global model with the clients
    for client in clients:
        client['model'].load_state_dict(global_model.state_dict())
        
# if (args.save_model):
#     torch.save(global_model.state_dict(), "FedAvg.pt")